In [1]:
import deeplabcut
path_config_file="C:\\Users\\Brian\\DeepLabCut\\examples\\AllVideoAttempt-BRI-2020-04-20\\config.yaml"
video=['F:\\Users\\Gittis\\Dropbox\\Gittis Lab Data\\OptoBehavior\\GPe\\Naive\\A2A\\Ai32\\Bilateral\\10x10\\AG5363_2_BI121719\\Trial    38.mpg',
       'F:\\Users\\Gittis\\Dropbox\\Gittis Lab Data\\OptoBehavior\\GPe\\Naive\\A2A\\Ai32\\Bilateral\\10x10\\AG5477_4_BI022520\\Trial   540.mpg',
       'F:\\Users\\Gittis\\Dropbox\\Gittis Lab Data\\OptoBehavior\\GPe\\Naive\\CAG\\Arch\\Bilateral\\10x10\\AG4766_10_BI021120\\Trial     2.mpg',
       'F:\\Users\\Gittis\\Dropbox\\Gittis Lab Data\\OptoBehavior\\GPe\\Naive\\CAG\\Arch\\Bilateral\\trig_r\\AG4700_5_BI090619\\Trial    83.mpg',
       'F:\\Users\\Gittis\\Dropbox\\Gittis Lab Data\\OptoBehavior\\Str\\Naive\\D1\\PSAM_PSEM\\Bilateral\\60min\\AG5232_10_BI111519\\Trial    25.mpg',
       'F:\\Users\\Gittis\\Dropbox\\Gittis Lab Data\\OptoBehavior\\Str\\Naive\\A2A\\ChR2\\Left\\5x30\\AG4700_6_BI083019\\Trial   368.mpg',
       'F:\\Users\\Gittis\\Dropbox\\Gittis Lab Data\\OptoBehavior\\GPe\\Naive\\PV\\Arch\\Bilateral\\zone_1\\AG4187_2_JS011519\\Trial    51.mpg'
      ]

In [ ]:
#Make a new project
task='AllVideoAttempt' # Enter the name of your experiment Task
experimenter='BRI' # Enter the name of the experimenter

# WARNING: Must run as administrator to have copy_videos = False (allow making symlinks)!!!!!
path_config_file=deeplabcut.create_new_project(task,experimenter,video,copy_videos=False) 


In [ ]:
#help(deeplabcut.create_new_project)
print('break test')
break
print('break test failed')

In [ ]:
#Add videos to an existing model / project:
new_video=['F:\\Users\\Gittis\\Dropbox\\Gittis Lab Data\\OptoBehavior\\SNr_and_Str\\Naive\\D1_D1\\Ai35_Ai35\\Bilateral\\10x30\\AG5313_4_BI120319\\Trial    35.mpg']
deeplabcut.add_new_videos(path_config_file,new_video,copy_videos=False)
[video.append(new) for new in new_video]

## Extract frames from videos 
A key point for a successful feature detector is to select diverse frames, which are typical for the behavior you study that should be labeled.

This function selects N frames either uniformly sampled from a particular video (or folder) ('uniform'). Note: this might not yield diverse frames, if the behavior is sparsely distributed (consider using kmeans), and/or select frames manually etc.

Also make sure to get select data from different (behavioral) sessions and different animals if those vary substantially (to train an invariant feature detector).

Individual images should not be too big (i.e. < 850 x 850 pixel). Although this can be taken care of later as well, it is advisable to crop the frames, to remove unnecessary parts of the frame as much as possible.

Always check the output of cropping. If you are happy with the results proceed to labeling.

In [ ]:
help(deeplabcut.extract_frames)

In [ ]:
#Automatically extract frames to label manually:
%matplotlib inline
deeplabcut.extract_frames(path_config_file,algo = 'kmeans') 

""" Alternate method:
#SELECT RARE EVENTS MANUALLY:
%gui wx
deeplabcut.extract_frames(path_config_file,'manual')
"""



In [ ]:
# stopped at 83
# PST: Add list of points that will be labeled to the .yaml configuration file
deeplabcut.label_frames(path_config_file) #Note: this seems to hang upon exit (if not saved first?), or even after successful save
#deeplabcut.check_labels(path_config_file)

In [ ]:
init=a['scorer']
csv_path=r"C:\Users\Brian\DeepLabCut\examples\AllVideoAttempt-BRI-2020-04-20\labeled-data\Trial     2"
csv_fn="\\CollectedData_%s.csv" % init 
print(csv_path+csv_fn)

In [ ]:
# Possibly important: make sure order of columns in 
import yaml
import csv
import pandas as pd
from os import listdir
with open(path_config_file, 'r') as stream:
    a=yaml.safe_load(stream)
col_order=a['bodyparts']
init=a['scorer']
base_path=r'C:\Users\Brian\DeepLabCut\examples\AllVideoAttempt-BRI-2020-04-20\labeled-data'
dirs=listdir(base_path)
for d in dirs:
    if 'labeled' not in d:
        csv_path="%s\\%s" % (base_path,d)
        csv_fn="\\CollectedData_{}.csv".format(init)
        print("Analyzing %s" % csv_path + csv_fn)
        df=pd.read_csv(csv_path + csv_fn,header=0)
        cols=list(df)
        for col in cols:
            if 'scorer' not in col:
                body_part=df[col][0]
                xy=df[col][1]
                for ind,part in enumerate(col_order,start=1):
                    if body_part == part:                
                        new_col_ind=(ind-1)*2+1
                        if 'y' in xy:
                            new_col_ind +=1
                        #print('%s moved to %d' % (body_part+xy,new_col_ind))
                        df.insert(new_col_ind,col,df.pop(col))      
                        df.to_csv(csv_path + '\\CollectedData_BRI.csv',index=False)

In [ ]:
col

In [ ]:
from deeplabcut.utils import auxiliaryfunctions
from deeplabcut.utils import conversioncode
import pdb

conversioncode.convertcsv2h5(path_config_file,userfeedback=False)



In [ ]:
pdb.pm()

In [ ]:
deeplabcut.check_labels(path_config_file)

In [ ]:
help(deeplabcut.create_training_dataset)

In [ ]:
deeplabcut.create_training_dataset(path_config_file)

In [ ]:
help(deeplabcut.train_network)

In [11]:
# One full iteration cycle:
deeplabcut.create_training_dataset(path_config_file)
vid="C:\\Users\\Brian\\DeepLabCut\\examples\\AllVideoAttempt-BRI-2020-04-20\\videos"
deeplabcut.train_network(path_config_file,maxiters=1000000) #Train for 1,000,000 iterations max
deeplabcut.evaluate_network(path_config_file, plotting=False)
deeplabcut.analyze_videos(path_config_file,[vid],videotype='.mpg',batchsize=16) #Optimize batchsize or no batch?
deeplabcut.extract_outlier_frames(path_config_file,[vid], videotype='.mpg',outlieralgorithm='jump',automatic=True) #How to not require input dialog?
deeplabcut.create_labeled_video(path_config_file,[vid], videotype='.mpg',filtered=False)

labeled-data\Trial    25\img091959.png C:\Users\Brian\DeepLabCut\examples\AllVideoAttempt-BRI-2020-04-20\dlc-models\iteration-10\AllVideoAttemptApr20-trainset95shuffle1\train
labeled-data\Trial    38\img05849.png C:\Users\Brian\DeepLabCut\examples\AllVideoAttempt-BRI-2020-04-20\dlc-models\iteration-10\AllVideoAttemptApr20-trainset95shuffle1\train
labeled-data\Trial   368\img05980.png C:\Users\Brian\DeepLabCut\examples\AllVideoAttempt-BRI-2020-04-20\dlc-models\iteration-10\AllVideoAttemptApr20-trainset95shuffle1\train
labeled-data\Trial    38\img06609.png C:\Users\Brian\DeepLabCut\examples\AllVideoAttempt-BRI-2020-04-20\dlc-models\iteration-10\AllVideoAttemptApr20-trainset95shuffle1\train
labeled-data\Trial    38\img19360.png C:\Users\Brian\DeepLabCut\examples\AllVideoAttempt-BRI-2020-04-20\dlc-models\iteration-10\AllVideoAttemptApr20-trainset95shuffle1\train
labeled-data\Trial   540\img13000.png C:\Users\Brian\DeepLabCut\examples\AllVideoAttempt-BRI-2020-04-20\dlc-models\iteration-10\A

labeled-data\Trial     2\img18776.png C:\Users\Brian\DeepLabCut\examples\AllVideoAttempt-BRI-2020-04-20\dlc-models\iteration-10\AllVideoAttemptApr20-trainset95shuffle1\train
labeled-data\Trial    25\img051840.png C:\Users\Brian\DeepLabCut\examples\AllVideoAttempt-BRI-2020-04-20\dlc-models\iteration-10\AllVideoAttemptApr20-trainset95shuffle1\train
labeled-data\Trial    25\img044020.png C:\Users\Brian\DeepLabCut\examples\AllVideoAttempt-BRI-2020-04-20\dlc-models\iteration-10\AllVideoAttemptApr20-trainset95shuffle1\train
labeled-data\Trial     2\img01765.png C:\Users\Brian\DeepLabCut\examples\AllVideoAttempt-BRI-2020-04-20\dlc-models\iteration-10\AllVideoAttemptApr20-trainset95shuffle1\train
labeled-data\Trial    25\img012365.png C:\Users\Brian\DeepLabCut\examples\AllVideoAttempt-BRI-2020-04-20\dlc-models\iteration-10\AllVideoAttemptApr20-trainset95shuffle1\train
labeled-data\Trial    25\img044782.png C:\Users\Brian\DeepLabCut\examples\AllVideoAttempt-BRI-2020-04-20\dlc-models\iteration-1

labeled-data\Trial    83\img028278.png C:\Users\Brian\DeepLabCut\examples\AllVideoAttempt-BRI-2020-04-20\dlc-models\iteration-10\AllVideoAttemptApr20-trainset95shuffle1\train
labeled-data\Trial     2\img06281.png C:\Users\Brian\DeepLabCut\examples\AllVideoAttempt-BRI-2020-04-20\dlc-models\iteration-10\AllVideoAttemptApr20-trainset95shuffle1\train
labeled-data\Trial    83\img043700.png C:\Users\Brian\DeepLabCut\examples\AllVideoAttempt-BRI-2020-04-20\dlc-models\iteration-10\AllVideoAttemptApr20-trainset95shuffle1\train
labeled-data\Trial   540\img07187.png C:\Users\Brian\DeepLabCut\examples\AllVideoAttempt-BRI-2020-04-20\dlc-models\iteration-10\AllVideoAttemptApr20-trainset95shuffle1\train
labeled-data\Trial   540\img14495.png C:\Users\Brian\DeepLabCut\examples\AllVideoAttempt-BRI-2020-04-20\dlc-models\iteration-10\AllVideoAttemptApr20-trainset95shuffle1\train
labeled-data\Trial   540\img23359.png C:\Users\Brian\DeepLabCut\examples\AllVideoAttempt-BRI-2020-04-20\dlc-models\iteration-10\

labeled-data\Trial   540\img06619.png C:\Users\Brian\DeepLabCut\examples\AllVideoAttempt-BRI-2020-04-20\dlc-models\iteration-10\AllVideoAttemptApr20-trainset95shuffle1\train
labeled-data\Trial   540\img01951.png C:\Users\Brian\DeepLabCut\examples\AllVideoAttempt-BRI-2020-04-20\dlc-models\iteration-10\AllVideoAttemptApr20-trainset95shuffle1\train
labeled-data\Trial     2\img24172.png C:\Users\Brian\DeepLabCut\examples\AllVideoAttempt-BRI-2020-04-20\dlc-models\iteration-10\AllVideoAttemptApr20-trainset95shuffle1\train
labeled-data\Trial    38\img05365.png C:\Users\Brian\DeepLabCut\examples\AllVideoAttempt-BRI-2020-04-20\dlc-models\iteration-10\AllVideoAttemptApr20-trainset95shuffle1\train
labeled-data\Trial     2\img19846.png C:\Users\Brian\DeepLabCut\examples\AllVideoAttempt-BRI-2020-04-20\dlc-models\iteration-10\AllVideoAttemptApr20-trainset95shuffle1\train
labeled-data\Trial     2\img06470.png C:\Users\Brian\DeepLabCut\examples\AllVideoAttempt-BRI-2020-04-20\dlc-models\iteration-10\Al

labeled-data\Trial   540\img12363.png C:\Users\Brian\DeepLabCut\examples\AllVideoAttempt-BRI-2020-04-20\dlc-models\iteration-10\AllVideoAttemptApr20-trainset95shuffle1\train
labeled-data\Trial   368\img17199.png C:\Users\Brian\DeepLabCut\examples\AllVideoAttempt-BRI-2020-04-20\dlc-models\iteration-10\AllVideoAttemptApr20-trainset95shuffle1\train
labeled-data\Trial     2\img01322.png C:\Users\Brian\DeepLabCut\examples\AllVideoAttempt-BRI-2020-04-20\dlc-models\iteration-10\AllVideoAttemptApr20-trainset95shuffle1\train
labeled-data\Trial    25\img034810.png C:\Users\Brian\DeepLabCut\examples\AllVideoAttempt-BRI-2020-04-20\dlc-models\iteration-10\AllVideoAttemptApr20-trainset95shuffle1\train
labeled-data\Trial    51\img028993.png C:\Users\Brian\DeepLabCut\examples\AllVideoAttempt-BRI-2020-04-20\dlc-models\iteration-10\AllVideoAttemptApr20-trainset95shuffle1\train
labeled-data\Trial     2\img08575.png C:\Users\Brian\DeepLabCut\examples\AllVideoAttempt-BRI-2020-04-20\dlc-models\iteration-10\

...

Kmeans clustering ... (this might take a while)
Let's select frames indices: [33122, 94037, 86553, 116471, 439]


0it [00:00, ?it/s]

New video was added to the project! Use the function 'extract_frames' to select frames for labeling.
The outlier frames are extracted. They are stored in the subdirectory labeled-data\Trial    51.
Once you extracted frames for all videos, use 'refine_labels' to manually correct the labels.
Method  jump  found  841  putative outlier frames.
Do you want to proceed with extracting  5  of those?
If this list is very large, perhaps consider changing the paramters (start, stop, epsilon, comparisonbodyparts) or use a different method.
Frames from video Trial    83  already extracted (more will be added)!
Loading video...
Duration of video [s]:  4247.376466666667 , recorded @  29.97002997002997 fps!
Overall # of frames:  127294 with (cropped) frame dimensions: 
Kmeans-quantization based extracting of frames from 0.0  seconds to 4247.38  seconds.
Extracting and downsampling... 841  frames from the video.


841it [00:04, 189.57it/s]


Kmeans clustering ... (this might take a while)
Let's select frames indices: [57173, 34991, 32502, 18628, 1194]
New video was added to the project! Use the function 'extract_frames' to select frames for labeling.
The outlier frames are extracted. They are stored in the subdirectory labeled-data\Trial    83.
Once you extracted frames for all videos, use 'refine_labels' to manually correct the labels.
Analyzing all the videos in the directory
Starting %  . ['C:\\Users\\Brian\\DeepLabCut\\examples\\AllVideoAttempt-BRI-2020-04-20\\videos']
Labeled video already created.
Starting %  . ['C:\\Users\\Brian\\DeepLabCut\\examples\\AllVideoAttempt-BRI-2020-04-20\\videos']
Labeled video already created.
Starting %  . ['C:\\Users\\Brian\\DeepLabCut\\examples\\AllVideoAttempt-BRI-2020-04-20\\videos']
Labeled video already created.
Starting %  . ['C:\\Users\\Brian\\DeepLabCut\\examples\\AllVideoAttempt-BRI-2020-04-20\\videos']
Labeled video already created.
Starting %  . ['C:\\Users\\Brian\\DeepLabCu

In [12]:
vid="C:\\Users\\Brian\\DeepLabCut\\examples\\AllVideoAttempt-BRI-2020-04-20\\videos"
deeplabcut.analyze_videos(path_config_file,[vid],videotype='.mpg',batchsize=16) #Optimize batchsize or no batch?
#deeplabcut.extract_outlier_frames(path_config_file,[vid],videotype='.mpg', outlieralgorithm='jump',automatic=True) #How to not require input dialog?
deeplabcut.create_labeled_video(path_config_file,[vid],videotype='.mpg',filtered=False)

Config:
{'all_joints': [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9]],
 'all_joints_names': ['top_head',
                      'top_tail_base',
                      'top_body_center',
                      'snout',
                      'side_head',
                      'side_tail_base',
                      'side_left_fore',
                      'side_right_fore',
                      'side_left_hind',
                      'side_right_hind'],
 'batch_size': 16,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets\\iteration-10\\UnaugmentedDataSet_AllVideoAttemptApr20\\AllVideoAttempt_BRI95shuffle1.mat',
 'dataset_type': 'default',
 'deconvolutionstride': 2,
 'deterministic': False,
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': 'C:\\Users\\Brian\\anaconda3\\envs\\DLC-GPU\\lib\\site-packages\\deeplabcut\\pose_estimation_tensorflow\\models\\pretrained\\resnet_v1_50.ckpt',
 'intermediate_sup

Using snapshot-1000000 for model C:\Users\Brian\DeepLabCut\examples\AllVideoAttempt-BRI-2020-04-20\dlc-models\iteration-10\AllVideoAttemptApr20-trainset95shuffle1
Initializing ResNet
INFO:tensorflow:Restoring parameters from C:\Users\Brian\DeepLabCut\examples\AllVideoAttempt-BRI-2020-04-20\dlc-models\iteration-10\AllVideoAttemptApr20-trainset95shuffle1\train\snapshot-1000000


INFO:tensorflow:Restoring parameters from C:\Users\Brian\DeepLabCut\examples\AllVideoAttempt-BRI-2020-04-20\dlc-models\iteration-10\AllVideoAttemptApr20-trainset95shuffle1\train\snapshot-1000000
  0%|                                                                                        | 0/25186 [00:00<?, ?it/s]

Analyzing all the videos in the directory
Starting to analyze %  Trial   540.mpg
Loading  Trial   540.mpg
Duration of video [s]:  840.37 , recorded with  29.97 fps!
Overall # of frames:  25186  found with (before cropping) frame dimensions:  704 480
Starting to extract posture


25351it [06:49, 61.35it/s]                                                                                             

...

  0%|                                                                              | 26/25204 [00:00<01:40, 250.81it/s]

Starting %  . ['C:\\Users\\Brian\\DeepLabCut\\examples\\AllVideoAttempt-BRI-2020-04-20\\videos']
Loading  Trial    38.mpg and data.
25204
Duration of video [s]:  840.92 , recorded with  29.97 fps!
Overall # of frames:  25204 with cropped frame dimensions:  1280 512
Generating frames and creating video.


  0%|                                                                             | 26/134909 [00:00<08:52, 253.23it/s]

Starting %  . ['C:\\Users\\Brian\\DeepLabCut\\examples\\AllVideoAttempt-BRI-2020-04-20\\videos']
Loading  Trial    35.mpg and data.
134909
Duration of video [s]:  4501.52 , recorded with  29.97 fps!
Overall # of frames:  134909 with cropped frame dimensions:  1280 512
Generating frames and creating video.


100%|█████████████████████████████████████████████████████████████████████████| 134909/134909 [08:30<00:00, 264.13it/s]


In [ ]:
help(deeplabcut.analyze_videos)

In [ ]:
#deeplabcut.evaluate_network(path_config_file, plotting=False)
vid="C:\\Users\\Brian\\DeepLabCut\\examples\\AllVideoAttempt-BRI-2020-04-20\\videos"
deeplabcut.analyze_videos(path_config_file,[vid],batchsize=16,videotype='.mpg') #Optimize batchsize or no batch?
deeplabcut.create_labeled_video(path_config_file,[vid],videotype='.mpg',filtered=False)

In [ ]:
#MANAUL frame extraction:
deeplabcut.extract_frames(path_config_file,'manual')

In [6]:
vid="C:\\Users\\Brian\\DeepLabCut\\examples\\AllVideoAttempt-BRI-2020-04-20\\videos"
deeplabcut.extract_outlier_frames(path_config_file,[vid],outlieralgorithm='jump',videotype='.mpg',automatic=True)

16it [00:00, 152.88it/s]

Analyzing all the videos in the directory
Method  jump  found  83  putative outlier frames.
Do you want to proceed with extracting  5  of those?
If this list is very large, perhaps consider changing the paramters (start, stop, epsilon, comparisonbodyparts) or use a different method.
Frames from video Trial   540  already extracted (more will be added)!
Loading video...
Duration of video [s]:  840.3728666666667 , recorded @  29.97002997002997 fps!
Overall # of frames:  25186 with (cropped) frame dimensions: 
Kmeans-quantization based extracting of frames from 0.0  seconds to 840.37  seconds.
Extracting and downsampling... 83  frames from the video.


83it [00:00, 151.30it/s]


Kmeans clustering ... (this might take a while)
Let's select frames indices: [3849, 18435, 20078, 16655, 23359]


0it [00:00, ?it/s]

...

Kmeans clustering ... (this might take a while)
Let's select frames indices: [14010, 13034, 10685, 6808, 15856]


0it [00:00, ?it/s]

New video was added to the project! Use the function 'extract_frames' to select frames for labeling.
The outlier frames are extracted. They are stored in the subdirectory labeled-data\Trial   368.
Once you extracted frames for all videos, use 'refine_labels' to manually correct the labels.
Method  jump  found  3860  putative outlier frames.
Do you want to proceed with extracting  5  of those?
If this list is very large, perhaps consider changing the paramters (start, stop, epsilon, comparisonbodyparts) or use a different method.
Frames from video Trial    25  already extracted (more will be added)!
Loading video...
Duration of video [s]:  3600.7001556443984 , recorded @  29.969726812947457 fps!
Overall # of frames:  107912 with (cropped) frame dimensions: 
Kmeans-quantization based extracting of frames from 0.0  seconds to 3600.7  seconds.
Extracting and downsampling... 3860  frames from the video.


3860it [00:17, 216.01it/s]


Kmeans clustering ... (this might take a while)
Let's select frames indices: [2619, 27602, 60055, 47904, 85005]
New video was added to the project! Use the function 'extract_frames' to select frames for labeling.
The outlier frames are extracted. They are stored in the subdirectory labeled-data\Trial    25.
Once you extracted frames for all videos, use 'refine_labels' to manually correct the labels.


In [ ]:
deeplabcut.extract_outlier_frames?

In [9]:
#Manually refine:
deeplabcut.refine_labels(path_config_file)

Windows
Checking labels if they are outside the image
A training dataset file is already found for this video. The refined machine labels are merged to this data!
Closing... The refined labels are stored in a subdirectory under labeled-data. Use the function 'merge_datasets' to augment the training dataset, and then re-train a network using create_training_dataset followed by train_network!


In [10]:
#If above successful:
deeplabcut.merge_datasets(path_config_file)

Merged data sets and updated refinement iteration to 10.
Now you can create a new training set for the expanded annotated images (use create_training_dataset).


## Create labeled video
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

THIS HAS MANY FUN OPTIONS! 

``deeplabcut.create_labeled_video(config, videos, videotype='avi', shuffle=1, trainingsetindex=0, filtered=False, save_frames=False, Frames2plot=None, delete=False, displayedbodyparts='all', codec='mp4v', outputframerate=None, destfolder=None, draw_skeleton=False, trailpoints=0, displaycropped=False)``

So please check:

In [ ]:
video[1] + video[4]

In [ ]:
#When satisfied with quality of labeling, create example video:
#videofile_path='F:\\Users\\Gittis\\Dropbox\\Gittis Lab Data\\OptoBehavior\\Str\\Naive\\D1\\PSAM_PSEM\\Bilateral\\60min\\AG5232_10_BI111519\\Trial    25.mpg'

#deeplabcut.filterpredictions(path_config_file,[video[2]],filtertype='median',windowlength=9)
deeplabcut.create_labeled_video(path_config_file,video,filtered=False)


In [ ]:
#help(deeplabcut.create_labeled_video)
help(deeplabcut.filterpredictions)